In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gdown

## Loading and cleaning

In [46]:
# Loading data from GDrive.

# Data with BDL merged,
file_id = "14VMCwHF2qoezvXykUvjfMNM1a573gQ8M"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
df = pd.read_csv(file_extension, sep=",")

Downloading...
From (original): https://drive.google.com/uc?id=14VMCwHF2qoezvXykUvjfMNM1a573gQ8M
From (redirected): https://drive.google.com/uc?id=14VMCwHF2qoezvXykUvjfMNM1a573gQ8M&confirm=t&uuid=f3e4b91c-42e5-4b0c-b9b0-49b1e81f5090
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 511M/511M [00:10<00:00, 48.4MB/s] 
C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\2459605807.py:8: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_extension, sep=",")


In [47]:
# Changing "," to "." in numerical variables.
comma_cols_to_replace = {
    'mediana_wieku': 'bdl_age_median',
    'wydatki_powiat': 'bdl_district_expenses',
    'wynagrodzenia': 'bdl_average_salary'
}

# Replace commas with dots and convert to float.
for old_col, new_col in comma_cols_to_replace.items():
    df[new_col] = df[old_col].str.replace(',', '.', regex=False).astype(float)

df.drop(columns=comma_cols_to_replace.keys(), inplace=True)

In [48]:
# Other columns to rename.
col_names_to_replace = {
    'cena_mieszkan': 'bdl_avg_house_pricing',
    'saldo_migracji': 'bdl_migration_coefficient',
    'wsp_feminizacji': 'bdl_feminization_coefficient'
}

for old_col, new_col in col_names_to_replace.items():
    df[new_col] = df[old_col].astype(float)
    df.drop([old_col], axis=1, inplace=True)

## Feature engineering

In [49]:
df['filled_vacancies_per_all'] = df['result'] / df['vacancies']

In [50]:
df['offer_salary_mentioned'] = df['offer_salary'].notna().astype(int)

In [51]:
# Variable correlation
numerical_columns_df = df[[
    'work_time',
    'ad_views',
    'year',
    'applicants',
    'education_level',
    'salary_for_position',
    'women_percentage',
    'bdl_feminization_coefficient',
    'fluctuation',
    'employment_contract',
    'bdl_avg_house_pricing',
    'bdl_age_median',
    'bdl_migration_coefficient',
    'bdl_district_expenses',
    'bdl_average_salary',
    'result'
]]


correlation = numerical_columns_df.corr()['result']

# Display the correlation matrix
print(correlation)

work_time                      -0.006045
ad_views                        0.228746
year                           -0.082010
applicants                      0.328321
education_level                -0.171731
salary_for_position             0.011918
women_percentage                0.025791
bdl_feminization_coefficient   -0.011355
fluctuation                    -0.065407
employment_contract             0.156612
bdl_avg_house_pricing          -0.090104
bdl_age_median                  0.020876
bdl_migration_coefficient      -0.056334
bdl_district_expenses          -0.080232
bdl_average_salary             -0.022229
result                          1.000000
Name: result, dtype: float64


In [52]:
# Dropping cols unnecessary for statistical analyses.
string_cols_for_temporary_dropping = [
    'ad_id',
    'institution',
    'institution_address',
    'workplace',
    'department',
    'job_field',
    'position_category',
    'responsibilities',
    'requirements',
    'offer_salary',
    'Unnamed: 0',
    'date_announced',
    'nice_to_have',
    'institution_address',
    'workplace',
    'department',
    'city',
    'position'
]
df_numerical = df.drop(string_cols_for_temporary_dropping, axis=1)

In [53]:
# Aggregating data to district-level.
df_numerical_district_level = df_numerical.groupby(['district', 'voivodeship', 'year']).agg('mean').reset_index()
df_numerical_district_level.head(100)

,district,voivodeship,year,result,education_level,work_time,vacancies,ad_views,advertised_institution,applicants,...,fluctuation,employment_contract,bdl_age_median,bdl_district_expenses,bdl_average_salary,bdl_avg_house_pricing,bdl_migration_coefficient,bdl_feminization_coefficient,filled_vacancies_per_all,offer_salary_mentioned
0,1,2,2017,1.000000,2.000000,1.000000,1.000000,2757.333333,NaN,NaN,...,NaN,NaN,NaN,990.41,1994.070,2824.0,14.0,106.0,1.000000,1.0
1,1,2,2018,0.750000,2.333333,1.000000,1.000000,2579.416667,NaN,10.080777,...,0.110491,213.601818,41.1,1137.92,2150.930,2818.0,35.0,106.0,0.750000,1.0
2,1,2,2019,0.750000,2.083333,1.000000,1.000000,2503.916667,1.000000,4.353977,...,0.061550,123.644000,41.5,1091.65,2316.520,3566.0,-8.0,106.0,0.750000,1.0
3,1,2,2020,0.909091,2.090909,1.000000,1.000000,2258.363636,1.000000,12.497335,...,0.039564,547.950000,42.1,1248.52,2488.225,4188.0,-20.0,106.0,0.909091,1.0
4,1,2,2021,0.800000,1.700000,1.000000,1.100000,1729.200000,0.666667,11.856658,...,0.205618,501.230000,42.4,1290.95,2776.470,4467.0,53.0,106.0,0.750000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,30,2021,0.260870,2.826087,1.000000,1.043478,1192.478261,0.173913,1.951240,...,0.105135,164.449565,NaN,NaN,NaN,NaN,NaN,NaN,0.239130,1.0
96,1,30,2022,0.272727,2.772727,0.965909,1.000000,1031.227273,0.090909,1.318415,...,0.008238,20.625909,NaN,NaN,NaN,NaN,NaN,NaN,0.272727,1.0
97,1,30,2023,0.777778,2.222222,1.000000,1.000000,991.666667,0.666667,7.697741,...,0.092165,411.698750,NaN,NaN,NaN,NaN,NaN,NaN,0.777778,1.0
98,1,32,2017,1.000000,1.000000,1.000000,1.000000,2631.000000,NaN,NaN,...,NaN,NaN,NaN,1162.28,1919.850,2037.0,-93.0,104.0,1.000000,1.0
